In [2]:
import pandas as pd
import numpy as np

### FUNCTION INVENTORY ###

In [3]:
def verticalMask(df, drop_cols = ['Program', 'Level', 'Number']):
    '''Asssuming the dataframe only contains 1 geography,
    iterrate through each column to perform vertical masking
    :param df: Dataframe
    :param drop_cols: list of columns to NOT be masked/ignored
    '''
    d = df.copy()
    #Iteratate through every report column
    cols = list(set(d.columns.tolist()) - set(drop_cols))
    
    for c in cols:
        print("...Processing column",c)
        #Only mask if a single number in that column is -999
#         print(np.sum(d.loc[d[c] < 0, c]))
        if sum(d.loc[d[c] < 0, c]) == -999: 
            next_min_a = np.min(d.loc[d[c] >= 0 , c])
            next_min_b = np.min(d.loc[d[c] > 0 , c])
            #Only allow 0 if is true next smallest
            print("min_a",next_min_a,"min_b",next_min_b)
            next_min = max(next_min_a, next_min_b )
#             next_min_idx = np.where(d[c].values  ==next_min)[0][0] 
            col_values = d[c]
#             print("col_values",col_values)
            next_min_idx = col_values[col_values == next_min].index.tolist()[0]
            print("The next smallest value for column {} is at index {} of value {}".format(c,next_min_idx,next_min))
            #Secondary mask 
            d.loc[next_min_idx, c] = -100
    return d

In [4]:
def secondMask(row):
    '''
    Pass a vector in, mask if only one -999 value
    :param row: a numpy vector
    '''
    if len(row[row ==  -999]) == 1:
#         print("Row before:\n", row)
        next_min_a = np.min(row[row >= 0])
        next_min_b = np.min(row[row > 0])
        #Only allow 0 if it is true next smallest
        next_min  = max(next_min_a, next_min_b) 
        
        next_min_idx = np.where(row==next_min)[0][0]
#         next_min_idx = row[row==next_min].index.tolist()[0]
        print("Next smallest at index {} of value {}".format(next_min_idx, next_min))
        #Mask
        row[next_min_idx] = -100
#         print("Row after:\n", row)
    return row

In [5]:
def horizontalMask(df, grp:dict):
    '''
    Leveraging the secondMask function, change the 
    group of related column names, then perform masking 
    within the parameters for each row in group. 
    :param df: Pandas dataframe
    :grp: dictionary of gorup names
    '''
    d = df.copy()
    for k,v in grp.items():
        d.loc[:,v] = d.loc[:,v].apply(secondMask, axis=1)
    return d

In [6]:
def splitReport(df:pd.DataFrame, geo=["State Assembly Total","State Senate Total"
                                      ,"U.S. Congress Total"
                                     ,"County Total"]):
    '''
    Accept a DF that is the original Excel report,
    split into N sub-reports based on geographies as 
    lited by the geo parameters
    '''
    #We can identify each section of the report here...
    indices = np.zeros(len(geo),dtype='int')
    split_dfs = []
    for idx in range(len(geo)):
        print("GEOGRAPHY level:", geo[idx])
        indices[idx] = df.index[df.Number == geo[idx]].tolist()[0]
        if idx == 0:
            split_dfs.append(df.iloc[:indices[idx]+1])
        else:
            split_dfs.append(df.iloc[indices[idx-1]+1:indices[idx]+1])
    return split_dfs

In [7]:
def batchMask(df, grp):
    masked_df = splitReport(df)
    final_df = pd.DataFrame()
    for i in range(len(masked_df)):
        #Drop index
        #Horizontal masking
        masked_df[i] = horizontalMask(masked_df[i], grp )
        #Vertical masking
        masked_df[i] = verticalMask(masked_df[i])   
        #Concatenate
        final_df = pd.concat([final_df, masked_df[i]], ignore_index=True)
        
#     #Finally, change al -999 and -100 into *1 and *2, repsetively;
    final_df = final_df.replace(-999,"*1")
    final_df = final_df.replace(-100,"*2")

#     if ! assert df.shape == final_df.shape: 
#         print("Final dataframe does NOT preserve original shape!!")
    final_df = final_df.reset_index(drop=True)
    return final_df

In [42]:
def createGroupdict(prog):
    group_dict = dict()
    
    #Populate dictionary
    group_dict["race"] =  ['Asian_PI','Black', 'Hispanic', 'Native American/Other/Unknown',"White"]
    group_dict["prog"] = ['1 program', '2 programs', '3 programs','4 programs', '5+ programs']
    group_dict["agy"] = ['1 department', '2 departments','3 departments', '4 departments']
    #Variable fields...
    group_dict["age1"] = ['17 and Under', '18 and Over']
    group_dict["age2"] = ['18-64','65 and Over']
    group_dict["gender"] = ['Female','Male', 'Unknown/Other Gender']
    #Special modification
    if prog.upper() == 'WIC': 
        group_dict["gender"] = ['Female','Male']
        group_dict["age1"]  = ['less than 19', '20 to 24', '25 to 29', '30 to 34','35 and over']
        group_dict["age2"]  = ['age 0', 'age 1', 'age 2', 'age 3', 'age 4','5 to 19','20 and over']
    elif prog.upper() == "CWS_CMS" :
#         group_dict["age1"] =  ['17 and Under','18 and Over']
#         group_dict["age2"] = ['18-64', '65 and Over']
        group_dict["fcage"] = ['age 0', '1 to 2', '3 to 5','6 to 10', '11 to 15', '16 to 17', '18 to 20']
    elif prog.upper() == "FC" :
#         group_dict["age1"] =  ['17 and Under','18 and Over']
        group_dict['age2'] = ["18-64"]
        group_dict["fcage"] = ['age 0', '1 to 2', '3 to 5','6 to 10', '11 to 15', '16 to 17', '18 to 20']
    elif prog.upper() =='FPACT':
        group_dict["gender"] = ['Female','Male']

    return group_dict 

## PRODUCTION TEST

In [50]:
prog = "Medical"
d = pd.read_excel(prog+"_Dashboard_16.xlsx")
d.columns

Index(['Program', 'Level', 'Number', 'Person', 'ACA', 'FPACT', 'CalFresh',
       'CalWorks', 'IHSS', 'Child Welfare', 'Foster Care', 'DDS', 'WIC',
       'White', 'Hispanic', 'Black', 'Asian_PI',
       'Native American/Other/Unknown', 'Female', 'Male',
       'Unknown/Other Gender', '17 and Under', '18 and Over', '18-64',
       '65 and Over', '1 program', '2 programs', '3 programs', '4 programs',
       '5+ programs', '1 department', '2 departments', '3 departments',
       '4 departments'],
      dtype='object')

In [51]:
group_dict = createGroupdict(prog)

In [52]:
r = batchMask(d, group_dict)

GEOGRAPHY level: State Assembly Total
GEOGRAPHY level: State Senate Total
GEOGRAPHY level: U.S. Congress Total
GEOGRAPHY level: County Total
Next smallest at index 1 of value 77968
Next smallest at index 1 of value 6287983
...Processing column 2 programs
...Processing column WIC
...Processing column 5+ programs
...Processing column 17 and Under
...Processing column Foster Care
...Processing column Black
...Processing column CalWorks
...Processing column DDS
...Processing column Unknown/Other Gender
...Processing column FPACT
...Processing column Person
...Processing column 1 program
...Processing column 2 departments
...Processing column Female
...Processing column Male
...Processing column 3 programs
...Processing column 1 department
...Processing column White
...Processing column Child Welfare
...Processing column CalFresh
...Processing column Native American/Other/Unknown
...Processing column 4 programs
...Processing column Asian_PI
...Processing column 3 departments
...Processing c

In [47]:
r.head()

,Program,Level,Number,Person,ACA,FPACT,CalFresh,CalWorks,IHSS,Child Welfare,...,65 and Over,1 program,2 programs,3 programs,4 programs,5+ programs,1 department,2 departments,3 departments,4 departments
0,MediCal,State Assembly,-9,100375,45501,4028,38230,8480,14,3297,...,4099,51581,35395,9879,3145,375,53742,40582,5763,288
1,MediCal,State Assembly,District 01,147143,45707,4168,61867,16036,65,3716,...,9620,72531,50791,18186,4988,647,74593,62318,9887,345
2,MediCal,State Assembly,District 02,159891,53319,7036,62757,13798,60,4055,...,9895,81489,54981,18392,4427,602,85595,64278,9574,444
3,MediCal,State Assembly,District 03,195691,56512,6179,83062,24627,121,4511,...,13133,93693,66072,27344,7666,916,96901,83344,14853,593
4,MediCal,State Assembly,District 04,133658,40147,5663,46677,10044,53,2006,...,9840,72078,43856,14293,3108,323,75505,49981,7802,370


In [53]:
r.to_excel("Masked_"+prog+"_Dashboard_16.xlsx",index=False)

In [54]:
################# VERIFYING CODE ####################

In [55]:
r.loc[(r.Level == "County") & (r.White==17), ["Asian_PI","Black","Hispanic","Native American/Other/Unknown","White"]]

,Asian_PI,Black,Hispanic,Native American/Other/Unknown,White


In [31]:
d.loc[(d.Level == "County") & (d.White==17), ["Asian_PI","Black","Hispanic","Native American/Other/Unknown","White"]]

,Asian_PI,Black,Hispanic,Native American/Other/Unknown,White
